<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Цели" data-toc-modified-id="Цели-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Цели</a></span></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Создание-и-обучение-модели" data-toc-modified-id="Создание-и-обучение-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Создание и обучение модели</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span><ul class="toc-item"><li><span><a href="#Комментраий-по-проекту" data-toc-modified-id="Комментраий-по-проекту-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Комментраий по проекту</a></span></li></ul></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп» с использованием BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

## Цели

Научить модель определять позитивный ли комментарий или нет

## Подготовка

Скачиваем и импортируем все требуемые библиотеки

In [1]:
pip install --upgrade scikit-learn -q 

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade scipy -q 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.0 requires numpy<1.23,>=1.18, but you have numpy 1.26.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade numpy -q

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade pandas -q 

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install --upgrade bottleneck -q --user

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install transformers -q

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install pymystem3 -q

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install torch -q

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
     |████████████████████████████████| 7.0 MB 1.5 MB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 757.3 MB 5.5 kB/s eta 0:00:0115��█████████████▌              | 414.0 MB 19.4 MB/s eta 0:00:18██████████████▉           | 494.3 MB 15.7 MB/s eta 0:00:17     |█████████████████████████▌      | 604.5 MB 60.7 MB/s eta 0:00:03
     |████████████████████████████████| 166.0 MB 36 kB/s  eta 0:00:011
     |████████████████████████████████| 121.6 MB 30 kB/s s eta 0:00:01     |███████████████████████████████▌| 119.5 MB 11.7 MB/s eta 0:00:01
     |████████████████████████████████| 23.7 MB 61.8 MB/s eta 0:00:01                    | 563 kB 61.8 MB/s eta 0:00:01
     |████████████████████████████████| 99 kB 9.5 MB/s  eta 0:00:01
     |████████████████████████████████| 196.0 MB 21 kB/s  eta 0:00:012
     |████████████████████████████████| 14.1 MB 76.4 MB/s eta 0:00:01
  

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, \
                                    cross_val_score, TimeSeriesSplit
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, \
                                  MinMaxScaler, RobustScaler, LabelEncoder, PolynomialFeatures
from sklearn.impute import SimpleImputer 
from sklearn.metrics import roc_auc_score, f1_score, fbeta_score, make_scorer, mean_squared_error, accuracy_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.base import clone
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 

# from pymystem3 import Mystem
import re 
from nltk.corpus import stopwords as nltk_stopwords

import nltk
import string

import torch
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, AdamW

from tqdm import notebook 

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import nltk

RANDOM_STATE = 42
np.random.seed(42)

In [11]:
torch.cuda.is_available()

False

In [12]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [13]:
nltk.download('stopwords') 
nltk.download('wordnet')
nltk.download("omw-1.4")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Импортируем базу данных

In [14]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [15]:
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [17]:
df['text'].duplicated().sum()

0

Дубликатов нет

In [18]:
df['toxic'].value_counts()

toxic
0    143106
1     16186
Name: count, dtype: int64

Избавляемся от символов, отдельных букв, слов с числами, а также переводит текст в нижний регистр.

In [19]:
def clean_text(text):
    '''
    This function removes punctuation, words containing numbers
    as well as making the whole text lower-case
    '''
    text = text.replace('\n', ' ')
    text = text.lower()
    text = text.encode("ascii", errors="ignore").decode()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\n', '', text)
    text= text.split()
    for word in text:
        if len(word) <=1:
            text.remove(word)
    text = ' '.join([str(elem) for elem in text])    
    return text


In [20]:
df['text'] = df['text'].apply(clean_text)
df.head()

,text,toxic
0,explanation why the edits made under my userna...,0
1,daww he matches this background colour im seem...,0
2,hey man im really not trying to edit war its j...,0
3,more cant make any real suggestions on improve...,0
4,you sir are my hero any chance you remember wh...,0


Удаляем строки с пустым тестом

In [21]:
df = df[df['text'] != '']

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159252 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159252 non-null  object
 1   toxic   159252 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


159252 количество строк. Оно не делится ни на какое удобное число, поэтому уберем 52 строки и сделаем батчи размером 100. 52 значения это несравнимо мало в сравнении с 159 тысячами, так что их можно спокойно убрать.

In [23]:
drop_indices = np.random.choice(df.index, 52, replace=False)
df = df.drop(drop_indices).reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159200 entries, 0 to 159199
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159200 non-null  object
 1   toxic   159200 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Функция для леммматизации. Она нам вроде не пригодится, но пока лучше просто оставим в закомментированном виде.


```m = WordNetLemmatizer()

def lemmatize(text):
    res = []
    word_tokens = word_tokenize(text)
    for i in word_tokens:
        res.append(m.lemmatize(i, pos="v"))
    return(' '.join(res))

df['text'] = df['text'].apply(lemmatize)

df.head()  
```

Токенизируем тексты

In [24]:
tokenizer = BertTokenizer.from_pretrained('unitary/toxic-bert')

tokenized = df['text'].apply(
    lambda x: tokenizer.encode(x, max_length = 100, add_special_tokens=True, truncation =True))
    
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/174 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/811 [00:00<?, ?B/s]

In [25]:
padded.shape

(159200, 100)

## Создание и обучение модели

In [26]:
model = BertModel.from_pretrained('unitary/toxic-bert')

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
%%time
batch_size = 100 # для примера возьмем такой батч, где будет всего две строки датасета
embeddings = []
model.to(device)   # закидываем модель на GPU
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).to(device) # закидываем тензор на GPU
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).to(device)

        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)

        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy()) # перевод обратно на проц, чтобы в нумпай кинуть
        del batch
        del attention_mask_batch
        del batch_embeddings

  0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 21.3 s, sys: 4.2 s, total: 25.5 s
Wall time: 25.7 s


Оставим BERT без GPU на всякий случай.
```
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

```

In [35]:
features = np.concatenate(embeddings)
features = pd.DataFrame(features)
print(features.shape)

(159200, 768)


Сохраним в отлельный файл, чтобы не генерировать в соледующий раз заново

In [36]:
features.to_csv(r"features.csv", index=False)

Выгрузим данные из файла (в последний раз просто выгружал данные из файла, а не создавал заново, поэтому блоки выше не были запущены).

In [ ]:
features = pd.read_csv('features.csv')

Деление на тренировочную и тестовую выборки.

In [37]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159200 entries, 0 to 159199
Columns: 768 entries, 0 to 767
dtypes: float32(768)
memory usage: 466.4 MB


features = features.astype('float16') # матрица не влезла в оперативную память компьютера, поэтому пришлось уменьгшить точность

In [38]:
X_train, X_test, y_train, y_test = train_test_split(features, df['toxic'], test_size=0.1, random_state=RANDOM_STATE)
X_num_col = list(X_train.select_dtypes(['int64', 'float64']).columns)

Создаем пайплайн для различных моделей.

In [41]:
pipe_final= Pipeline(
    [
        ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
    ]
)

param_grid_tree = [
    # словарь для модели DecisionTreeClassifier()
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': range(2, 10),
        'models__max_features': range(2, 10),
        'models__min_samples_leaf': range(2, 10),
        'models__min_samples_split': range(2, 10)
    }
]

randomized_search_tree = RandomizedSearchCV(
    pipe_final, 
    param_grid_tree, 
    cv=5,
    scoring='f1',
    n_jobs=-1,
    n_iter=10,
    random_state=RANDOM_STATE
)

randomized_search_tree.fit(X_train, y_train)

print('Лучшая модель и её параметры:\n\n', randomized_search_tree.best_estimator_)
print ('Метрика лучшей модели на тренировочной выборке:', randomized_search_tree.best_score_)

Лучшая модель и её параметры:

 Pipeline(steps=[('models',
                 DecisionTreeClassifier(max_depth=8, max_features=8,
                                        min_samples_leaf=8, min_samples_split=5,
                                        random_state=42))])
Метрика лучшей модели на тренировочной выборке: 0.8872321372809668


In [44]:

pipe_kneighbor = Pipeline(
    [
        ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
    ]
)

param_kneighbor = [
    {
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': range(2, 5) 
    },

]

randomized_search_kneighbor = GridSearchCV(
    pipe_kneighbor, 
    param_kneighbor, 
    cv=5,
    scoring='f1',
    n_jobs=-1
)

randomized_search_kneighbor.fit(X_train, y_train)

print('Лучшая модель и её параметры:\n\n', randomized_search_kneighbor.best_estimator_)
print ('Метрика лучшей модели на тренировочной выборке:', randomized_search_kneighbor.best_score_)

Лучшая модель и её параметры:

 Pipeline(steps=[('models', KNeighborsClassifier(n_neighbors=3))])
Метрика лучшей модели на тренировочной выборке: 0.8840933490779719


In [46]:

pipe_lr = Pipeline(
    [
        ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
    ]
)

param_grid_lr = [
    {
        'models': [LogisticRegression(
            random_state=RANDOM_STATE, 
            solver='liblinear'
        )],
        'models__C': range(1,2) 
    }

]

randomized_search_lr = GridSearchCV(
    pipe_lr, 
    param_grid_lr, 
    cv=5,
    scoring='f1',
    n_jobs=-1
)

randomized_search_lr.fit(X_train, y_train)

print('Лучшая модель и её параметры:\n\n', randomized_search_lr.best_estimator_)
print ('Метрика лучшей модели на тренировочной выборке:', randomized_search_lr.best_score_)

Лучшая модель и её параметры:

 Pipeline(steps=[('models',
                 LogisticRegression(C=1, random_state=42, solver='liblinear'))])
Метрика лучшей модели на тренировочной выборке: 0.9021982111751831


**Лучше всех справилась логичтисеская регрессия. 90.2% на тренировочной выборке. Проверим ее на тестовой.**

In [47]:
y_pred = randomized_search_lr.predict(X_test)
f1_score(y_test, y_pred)

0.9046839174582378

Отличный результат, такая модель нам подойдет.

## Выводы

1. Данные в хорошем состоянии их только пришлось обрабоать для обучения модели
2. Само обучение BERT заняло около 8 минут с использованием GPU. Был использован BERT уже подготовленный к определению токсичных комментариев.
3. Модель с лучшей метрикой f1 - это линейная регрессия с коэффициентом С = 1. Её значение - это 0.902 на тренировочной и 0.905 тестовой.